In [3]:
import pandas as pd
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier   #导入sklearn.neighbors模块中KNN类
from sklearn.linear_model import LogisticRegression
fname = '../data/sqlResult_1558435.csv'

In [5]:
content = pd.read_csv(fname, encoding='gb18030')
content_head = content.head(2000)
content_head = content_head[content_head['content'] != None ]
col_name = content_head.columns.tolist()

In [6]:
# content.loc[103]

In [7]:
col_name.insert(col_name.index('feature'),'Y')# 在 B 列前面插入

In [8]:
content_head = content_head.reindex(columns=col_name)

In [9]:
def load_stop_words(name):
    '''
    加载停用词
    :param name:停用词的本地存放文件
    :return: 停用词集合set()
    '''
    words = set()
    file = open(name, 'r', encoding="utf-8")
    for word in file.readlines():
        words.add(word.strip())
    file.close()
    return words

In [10]:
def precess_data_Y(arrLike,s_title):
    s = arrLike[s_title]
    ret = 0.0
    if s == '新华社':
        ret = 1.0
    return ret

def process_data_content(arrLike,s_content):
    STOP_WORDS = load_stop_words('../data/stop_words.txt')
    content = ''
    s = arrLike[s_content]
    if type(s)  == float:        # 去掉content为空的内容
        return content
    words = jieba.cut(s, cut_all=False)
    for word in words:
        if word not in STOP_WORDS:
            content += word + " "
    return content

In [11]:
content_head.loc[:,'Y'] = content_head.apply(precess_data_Y, axis = 1, s_title = 'source') 

In [12]:
content_head.loc[:,'content'] = content_head.apply(process_data_content, axis = 1, s_content = 'content') 

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/d1/3vczz6v51wg62zr7f6plwg280000gn/T/jieba.cache
Loading model cost 1.041 seconds.
Prefix dict has been built succesfully.


In [13]:
# content_head

In [14]:
vectorizer = TfidfVectorizer(encoding='gb18030')

In [15]:
X_train = vectorizer.fit_transform(content_head.loc[:,'content'], content_head.loc[:,'Y'])

In [16]:
 X_train, X_test, y_train, y_test = train_test_split(X_train, content_head.loc[:,'Y'], test_size=0.33, random_state=42)

## 1、KNN 模型

In [17]:
knn = KNeighborsClassifier()             #定义一个knn分类器对象
knn.fit(X_train, y_train)                #调用该对象的训练方法，主要接收两个参数：训练数据集及其样本标签

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [18]:
y_predict = knn.predict(X_test)  #调用该对象的测试方法，主要接收一个参数：测试数据集

In [85]:
i_index = 0
for x in y_predict:    
    if x > 0:print(x, i_index)
    i_index += 1
        

1.0 308


In [91]:
y_test[308:]

937     1.0
30      0.0
1091    0.0
1615    0.0
1229    0.0
974     0.0
1084    0.0
514     0.0
1550    0.0
1567    0.0
1810    0.0
1573    0.0
44      0.0
1686    0.0
168     0.0
303     0.0
990     1.0
555     0.0
1924    0.0
607     0.0
1618    0.0
757     0.0
438     0.0
582     0.0
1518    0.0
527     0.0
342     0.0
1736    0.0
367     0.0
185     0.0
       ... 
277     0.0
1846    0.0
358     0.0
1110    0.0
236     0.0
551     0.0
1867    0.0
1812    0.0
208     0.0
1386    0.0
1560    0.0
58      0.0
631     0.0
1961    0.0
1859    0.0
1816    0.0
363     0.0
505     0.0
1165    0.0
1121    0.0
1629    0.0
1216    0.0
1151    0.0
1124    0.0
1594    0.0
118     0.0
1249    0.0
1993    0.0
522     0.0
513     0.0
Name: Y, Length: 352, dtype: float64

In [19]:
score = knn.score(X_test,y_test,sample_weight=None)    #调用该对象的打分方法，计算出准确率

In [20]:
print('Accuracy:', score)

Accuracy: 0.9787878787878788


In [21]:
probility = knn.predict_proba(X_test)           #计算各测试样本基于概率的预测 

In [22]:
# y_test

In [23]:
# content_head.loc[964]

In [24]:
neighborpoint=knn.kneighbors(X_test[-1],5,False)  #计算与最后一个测试样本距离在最近的5个点，返回的是这些样本的序号组成的数组

In [25]:
print('neighborpoint of last test sample:',neighborpoint)

neighborpoint of last test sample: [[ 275 1102  984  343 1106]]


## 2、逻辑回归

In [26]:
clr = LogisticRegression(penalty = 'l2',solver='saga', verbose=1)
clr.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 1 seconds


/Users/yangbo/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='saga',
          tol=0.0001, verbose=1, warm_start=False)

In [27]:
y_predict = clr.predict(X_test)

In [28]:
probility = clr.predict_proba(X_test)           #计算各测试样本基于概率的预测 

In [29]:
score = clr.score(X_test,y_test,sample_weight=None)    #调用该对象的打分方法，计算出准确率

In [30]:
print('Accuracy:', score)

Accuracy: 0.9757575757575757


In [31]:
probility

array([[0.96976965, 0.03023035],
       [0.97552181, 0.02447819],
       [0.97950883, 0.02049117],
       ...,
       [0.97077825, 0.02922175],
       [0.97417998, 0.02582002],
       [0.9758766 , 0.0241234 ]])

## SVM支持向量机

In [32]:
from sklearn import svm
clf = svm.SVC(gamma='scale')

In [95]:
clf.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [40]:
result = clf.predict(X_test)

In [94]:
# for index, value in y_test.items():
#     if value >= 1:
#         print(index)
#         print(content_head.loc[index]['content'])
content_head.loc[937]['content']

'\u3000 \u3000 新华社 快讯 纽约商品交易所 黄金 期货市场 交投 活跃 ８ 黄金 期价 ２ ０ 日比前 交易日 下跌 ３ ． ２ 美元 收于 每盎司 １ ２ ４ ３ ． ５ 美元 跌幅 ０ ． ２ ６ ％ \r\n '

In [65]:
score = clf.score(X_test,y_test)

In [37]:
print('Accuracy:', score)

Accuracy: 0.9742424242424242


## Naive Bayes 朴素贝叶斯

In [96]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

In [106]:
gnb.fit(X_train.toarray(), y_train.values)

GaussianNB(priors=None, var_smoothing=1e-09)

In [107]:


score = gnb.score(X_test.toarray(), y_test.values)

In [109]:
gnb.predict(X_test.toarray())

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [108]:
print('Accuracy:', score)

Accuracy: 0.9787878787878788
